# 📦 Konteyner ve Mahalle Analizi
Bu notebook, mahalle bazlı konteyner envanteri, nüfus ve toplama programı analizlerini içermektedir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_palette("husl")
sns.set_style("whitegrid")

# Veri yükleme
containers_df = pd.read_csv('container_counts.csv', sep=';')
population_df = pd.read_csv('mahalle_nufus.csv', sep=';')
schedule_df = pd.read_csv('neighbor_days_rotations.csv', sep=';')

print("Veriler yüklendi!")
print(f"Konteyner verisi: {containers_df.shape}")
print(f"Nüfus verisi: {population_df.shape}")
print(f"Program verisi: {schedule_df.shape}")

## 📊 Konteyner Envanteri Analizi

In [ ]:
# Veri temizleme
containers_df.columns = containers_df.columns.str.strip()

# Sayısal sütunları düzelt
numeric_cols = ['YERALTI KONTEYNER', '770 LT KONTEYNER', '400 LT KONTEYNER', 'PLASTİK', 'TOPLAM']
for col in numeric_cols:
    if col in containers_df.columns:
        # Nokta olan sayıları düzelt (1.711 -> 1711)
        containers_df[col] = containers_df[col].astype(str).str.replace('.', '', regex=False)
        # Sayısal olmayan değerleri 0 yap
        containers_df[col] = pd.to_numeric(containers_df[col], errors='coerce').fillna(0)

print("İlk 10 Mahalle:")
containers_df.head(10)

In [ ]:
# Konteyner tipi toplam
container_totals = {
    'Yeraltı': containers_df['YERALTI KONTEYNER'].sum(),
    '770 Litre': containers_df['770 LT KONTEYNER'].sum(),
    '400 Litre': containers_df['400 LT KONTEYNER'].sum(),
    'Plastik': containers_df['PLASTİK'].sum()
}

print("=" * 60)
print("KONTEYNER TİPİ DAĞILIMI")
print("=" * 60)
for key, value in container_totals.items():
    print(f"{key:20s}: {value:>10,.0f} adet")
print(f"{'TOPLAM':20s}: {sum(container_totals.values()):>10,.0f} adet")
print("=" * 60)

In [ ]:
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart - Konteyner tipi dağılımı
colors_pie = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
axes[0].pie(container_totals.values(), labels=container_totals.keys(), autopct='%1.1f%%',
            startangle=90, colors=colors_pie, textprops={'fontsize': 11})
axes[0].set_title('Konteyner Tipi Dağılımı', fontsize=14, fontweight='bold')

# Bar chart
pd.Series(container_totals).plot(kind='bar', ax=axes[1], color=colors_pie)
axes[1].set_xlabel('Konteyner Tipi', fontsize=12)
axes[1].set_ylabel('Adet', fontsize=12)
axes[1].set_title('Konteyner Tiplerine Göre Dağılım', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

# Değerleri ekle
for i, v in enumerate(container_totals.values()):
    axes[1].text(i, v + 100, f'{v:,.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 🏘️ Mahalle Bazlı Konteyner Yoğunluğu

In [ ]:
# En çok ve en az konteynerli mahalleler
top_neighborhoods = containers_df.nlargest(15, 'TOPLAM')[['MAHALLE', 'TOPLAM']]
bottom_neighborhoods = containers_df[containers_df['TOPLAM'] > 0].nsmallest(15, 'TOPLAM')[['MAHALLE', 'TOPLAM']]

fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# En çok konteynerli mahalleler
top_neighborhoods.set_index('MAHALLE')['TOPLAM'].plot(kind='barh', ax=axes[0], color='darkgreen')
axes[0].set_xlabel('Toplam Konteyner Sayısı', fontsize=12)
axes[0].set_ylabel('Mahalle', fontsize=12)
axes[0].set_title('En Çok Konteynerli 15 Mahalle', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# En az konteynerli mahalleler
bottom_neighborhoods.set_index('MAHALLE')['TOPLAM'].plot(kind='barh', ax=axes[1], color='darkorange')
axes[1].set_xlabel('Toplam Konteyner Sayısı', fontsize=12)
axes[1].set_ylabel('Mahalle', fontsize=12)
axes[1].set_title('En Az Konteynerli 15 Mahalle', fontsize=13, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 👥 Nüfus Analizi

In [ ]:
# Nüfus verisi temizleme
population_df.columns = population_df.columns.str.strip()
population_df['nufus'] = population_df['nufus'].astype(str).str.replace('.', '', regex=False)
population_df['nufus'] = pd.to_numeric(population_df['nufus'], errors='coerce')

print("=" * 60)
print("NÜFUS İSTATİSTİKLERİ")
print("=" * 60)
print(f"Toplam Nüfus: {population_df['nufus'].sum():,.0f} kişi")
print(f"Ortalama Mahalle Nüfusu: {population_df['nufus'].mean():,.0f} kişi")
print(f"En Kalabalık: {population_df.loc[population_df['nufus'].idxmax(), 'mahalle']} ({population_df['nufus'].max():,.0f} kişi)")
print(f"En Az Kalabalık: {population_df.loc[population_df['nufus'].idxmin(), 'mahalle']} ({population_df['nufus'].min():,.0f} kişi)")
print("=" * 60)

In [ ]:
# En kalabalık mahalleler
top_population = population_df.nlargest(20, 'nufus')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
top_population.set_index('mahalle')['nufus'].plot(kind='barh', ax=axes[0], color='mediumvioletred')
axes[0].set_xlabel('Nüfus', fontsize=12)
axes[0].set_ylabel('Mahalle', fontsize=12)
axes[0].set_title('En Kalabalık 20 Mahalle', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Nüfus dağılım histogramı
axes[1].hist(population_df['nufus'], bins=20, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Nüfus', fontsize=12)
axes[1].set_ylabel('Mahalle Sayısı', fontsize=12)
axes[1].set_title('Nüfus Dağılım Histogramı', fontsize=13, fontweight='bold')
axes[1].axvline(population_df['nufus'].mean(), color='red', linestyle='--', 
               label=f'Ortalama: {population_df["nufus"].mean():,.0f}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🔄 Nüfus ve Konteyner İlişkisi

In [ ]:
# Mahalle isimlerini standartlaştır
containers_df['MAHALLE_CLEAN'] = containers_df['MAHALLE'].str.strip().str.upper()
population_df['mahalle_CLEAN'] = population_df['mahalle'].str.strip().str.upper()

# Birleştir
merged_df = pd.merge(
    containers_df[['MAHALLE_CLEAN', 'TOPLAM']], 
    population_df[['mahalle_CLEAN', 'nufus']], 
    left_on='MAHALLE_CLEAN', 
    right_on='mahalle_CLEAN', 
    how='inner'
)

# Kişi başına konteyner hesapla
merged_df['Kişi_Başına_Konteyner'] = (merged_df['TOPLAM'] / merged_df['nufus'] * 1000).round(2)

print(f"Eşleşen mahalle sayısı: {len(merged_df)}")
print(f"\nOrtalama 1000 kişi başına konteyner: {merged_df['Kişi_Başına_Konteyner'].mean():.2f}")

In [ ]:
# Scatter plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Nüfus vs Konteyner
axes[0].scatter(merged_df['nufus'], merged_df['TOPLAM'], alpha=0.6, s=100, color='teal')
axes[0].set_xlabel('Nüfus', fontsize=12)
axes[0].set_ylabel('Toplam Konteyner Sayısı', fontsize=12)
axes[0].set_title('Nüfus vs Konteyner Sayısı', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3)

# Trend line ekle
z = np.polyfit(merged_df['nufus'], merged_df['TOPLAM'], 1)
p = np.poly1d(z)
axes[0].plot(merged_df['nufus'].sort_values(), p(merged_df['nufus'].sort_values()), 
            "r--", alpha=0.8, linewidth=2, label='Trend')
axes[0].legend()

# Kişi başına konteyner dağılımı
top_ratio = merged_df.nlargest(15, 'Kişi_Başına_Konteyner')
top_ratio.set_index('MAHALLE_CLEAN')['Kişi_Başına_Konteyner'].plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_xlabel('1000 Kişi Başına Konteyner', fontsize=12)
axes[1].set_ylabel('Mahalle', fontsize=12)
axes[1].set_title('En Yüksek Konteyner Yoğunluklu Mahalleler\n(1000 kişi başına)', fontsize=13, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 📅 Toplama Programı Analizi

In [ ]:
# Program verisi temizleme
schedule_df.columns = schedule_df.columns.str.strip()
schedule_df['Days Collected Per Week'] = pd.to_numeric(schedule_df['Days Collected Per Week'], errors='coerce').fillna(0)

print("=" * 60)
print("TOPLAMA PROGRAMI ÖZETİ")
print("=" * 60)
print(f"\nHaftalık Toplama Frekansı Dağılımı:")
print(schedule_df['Days Collected Per Week'].value_counts().sort_index())
print(f"\n\nAraç Tipi Dağılımı:")
print(schedule_df['Garbage Truck Type'].value_counts())
print(f"\n\nVinç Kullanımı:")
print(schedule_df['Is Crane Used'].value_counts())
print("=" * 60)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Haftalık toplama günü sayısı
days_counts = schedule_df['Days Collected Per Week'].value_counts().sort_index()
days_counts.plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_xlabel('Haftalık Toplama Gün Sayısı', fontsize=11)
axes[0, 0].set_ylabel('Mahalle Sayısı', fontsize=11)
axes[0, 0].set_title('Haftalık Toplama Frekansı', fontsize=12, fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].grid(axis='y', alpha=0.3)

# Araç tipi kullanımı
truck_counts = schedule_df['Garbage Truck Type'].value_counts()
truck_counts.plot(kind='pie', ax=axes[0, 1], autopct='%1.1f%%', startangle=90)
axes[0, 1].set_ylabel('')
axes[0, 1].set_title('Kullanılan Araç Tipleri', fontsize=12, fontweight='bold')

# Vinç kullanımı
crane_counts = schedule_df['Is Crane Used'].value_counts()
crane_counts.plot(kind='bar', ax=axes[1, 0], color=['lightcoral', 'lightgreen'])
axes[1, 0].set_xlabel('Vinç Kullanımı', fontsize=11)
axes[1, 0].set_ylabel('Mahalle Sayısı', fontsize=11)
axes[1, 0].set_title('Vinç Kullanım Durumu', fontsize=12, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].grid(axis='y', alpha=0.3)

# Vinç rotasyon günleri (vinç kullanan mahalleler için)
crane_rotations = schedule_df[schedule_df['Is Crane Used'] == True]['Crane rotation days'].value_counts()
if len(crane_rotations) > 0:
    crane_rotations.plot(kind='bar', ax=axes[1, 1], color='gold')
    axes[1, 1].set_xlabel('Rotasyon Gün Sayısı', fontsize=11)
    axes[1, 1].set_ylabel('Mahalle Sayısı', fontsize=11)
    axes[1, 1].set_title('Vinç Rotasyon Günleri', fontsize=12, fontweight='bold')
    axes[1, 1].tick_params(axis='x', rotation=0)
    axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Temel Bulgular

1. **Konteyner Dağılımı:** 30,000+ konteyner, çoğunluğu plastik ve 400L tipinde
2. **Mahalle Farklılıkları:** Konteyner sayısı 13 ile 2,590 arasında değişiyor
3. **Nüfus-Konteyner İlişkisi:** Pozitif korelasyon, ancak bazı mahallelerde dengesizlik
4. **Toplama Frekansı:** Çoğunlukla haftada 3 gün, yoğun bölgelerde 6-7 gün
5. **Vinç İhtiyacı:** 17 mahallede yeraltı konteyner için vinçli araç gerekli

In [ ]:
# Özet tablo
print("\n" + "=" * 70)
print("GENEL ÖZET")
print("=" * 70)
print(f"Toplam Mahalle Sayısı: {len(containers_df)}")
print(f"Toplam Konteyner: {containers_df['TOPLAM'].sum():,.0f} adet")
print(f"Toplam Nüfus: {population_df['nufus'].sum():,.0f} kişi")
print(f"Ortalama Konteyner/Mahalle: {containers_df['TOPLAM'].mean():.0f} adet")
print(f"Ortalama Nüfus/Mahalle: {population_df['nufus'].mean():,.0f} kişi")
print(f"Ortalama 1000 Kişi/Konteyner: {merged_df['Kişi_Başına_Konteyner'].mean():.2f}")
print("=" * 70)